# Capstone Project - The Battle of Neighbourhoods (Week 1)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction](#introduction)
* [Data](#data)

## Introduction <a name="introduction"></a>

### Background
Cologne is Germany's fourth biggest city and has about 1 million inhabitants. It's located in the west of Germany, not far from the Dutch border. It's on the left bank of the river Rhine. Its famous Cathedral (Kölner Dom) is the seat of the Catholic Archbishop of Cologne and Cologne's landmark known internationally. Germany's only sport university is in Cologne and the Cologne Bonn Airport is Germany's seventh-largest airport.
I live in Cologne, that's why I choose Cologne for this final assignment. 
### Problem
The apartment market of Cologne has become really rough and it's hard to find an affordable apartment in one of the popular neighbourhoods. However, often the neighbourhoods are only popular and expensive because of their reputation. The infrastructure of other neighbourhoods within Cologne might be pretty similar and offer a similar living experience, but with cheaper apartment prices. 
### Interest
For people that either move to Cologne or look for a new apartment within Cologne (e.g. when moving together with a partner) it's interesting to find alternative to the popular and expensive neighbourhoods. Therefore, I compare and cluster the neighbourhoods to show potential alternatives and give some guidance to apartment-hunters in Cologne.

## Data <a name="data"></a>

### Preparation
At first, I import all the relevant libraries.

**Note:** I use a docker environment with an image that already contains all relevant packages, hence I don't need to install them.

In [1]:
import pandas as pd
import numpy as np
import requests
import io

import json # library to handle JSON files
from geopy.geocoders import Nominatim 
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry.polygon import LinearRing
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

import folium # map rendering library

print("All libraries imported :) Let's go!")

All libraries imported :) Let's go!


### Data acquisition and cleaning
#### Data sources
I use basically two data sources:
1. Open data provided by the city of Cologne itself: https://offenedaten-koeln.de: This page offers many data sets on all different kinds of information. It also offers vast geodata on Cologne's neighbourhoods including polygons of coordinates for each neighbourhood and socio-economic data on households, inhabitants, schools etc. per neighbourhood.
2. I will use the Foursquare API to get data on all different kinds of venues within each neighbourhood of Cologne to reflect the infrastructure of a neighbourhood with regard to restaurants, gyms, supermarkets and other categories. **Note: The assignment requires the usage of Foursquare. Otherwise, I would not use Foursquare because it's not so popular in Germany (or at least in Cologne) and I feel that it doesn't cover a lot of the venues.** 

##### 1. Location data for all neighbourhoods

In [2]:
# request the geodata from https://offenedaten-koeln.de/dataset/stadtteile

geodata = json.loads(requests.get('https://geoportal.stadt-koeln.de/arcgis/rest/services/Stadtgliederung_15/MapServer/1/query?where=objectid+is+not+null&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=4326&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&f=pjson').text)
geodata

{'displayFieldName': 'NAME',
 'fieldAliases': {'OBJECTID': 'OBJECTID',
  'NUMMER': 'Stadtteil (Nr.)',
  'NAME': 'Stadtteil',
  'NR_STADTBEZIRK': 'Stadtbezirk (Nr.)',
  'STADTBEZIRK': 'Stadtbezirk',
  'FLAECHE': 'Fläche (qm)',
  'LINK': 'LINK'},
 'geometryType': 'esriGeometryPolygon',
 'spatialReference': {'wkid': 4326, 'latestWkid': 4326},
 'fields': [{'name': 'OBJECTID',
   'type': 'esriFieldTypeOID',
   'alias': 'OBJECTID'},
  {'name': 'NUMMER',
   'type': 'esriFieldTypeString',
   'alias': 'Stadtteil (Nr.)',
   'length': 3},
  {'name': 'NAME',
   'type': 'esriFieldTypeString',
   'alias': 'Stadtteil',
   'length': 40},
  {'name': 'NR_STADTBEZIRK',
   'type': 'esriFieldTypeString',
   'alias': 'Stadtbezirk (Nr.)',
   'length': 1},
  {'name': 'STADTBEZIRK',
   'type': 'esriFieldTypeString',
   'alias': 'Stadtbezirk',
   'length': 40},
  {'name': 'FLAECHE', 'type': 'esriFieldTypeDouble', 'alias': 'Fläche (qm)'},
  {'name': 'LINK',
   'type': 'esriFieldTypeString',
   'alias': 'LINK',
 

In [3]:
# identify the keys

list(geodata.keys())

['displayFieldName',
 'fieldAliases',
 'geometryType',
 'spatialReference',
 'fields',
 'features']

In [4]:
#Normalize semi-structured JSON data into a flat table on the features

geo = json_normalize(geodata['features'])
geo.head()

,attributes.FLAECHE,attributes.LINK,attributes.NAME,attributes.NR_STADTBEZIRK,attributes.NUMMER,attributes.OBJECTID,attributes.STADTBEZIRK,geometry.rings
0,4592631,None,Godorf,2,211,7,Rodenkirchen,"[[[6.994359341598065, 50.85835989408827], [6.9..."
1,3692013,None,Lövenich,3,308,8,Lindenthal,"[[[6.835101297207516, 50.957260493877406], [6...."
2,3654710,None,Weiden,3,307,9,Lindenthal,"[[[6.849501526802686, 50.94220412958467], [6.8..."
3,7376623,None,Junkersdorf,3,306,10,Lindenthal,"[[[6.854198181651118, 50.94052346246431], [6.8..."
4,5599212,None,Widdersdorf,3,309,11,Lindenthal,"[[[6.851763526592514, 50.97718504868292], [6.8..."


The default names of the columns are not very handy, therefore I'll rename them accordingly. Furthermore, there are some columns that I don't need at all (e.g. attributes.LINK or attributes.NR_STADTBEZIRK). These will be dropped.

In [5]:
# drop irrelevant columns

geo.drop(['attributes.LINK'], 1, inplace = True)
geo.drop(['attributes.NR_STADTBEZIRK'], 1, inplace = True)
geo.drop(['attributes.OBJECTID'], 1, inplace = True)

# rename columns

geo.rename(columns = {"attributes.FLAECHE": "sqm","attributes.NAME":"neighbourhood", "attributes.NUMMER":"no.", 
                      "attributes.STADTBEZIRK":"borough"}, inplace = True)

# check dataframe
geo.head()


,sqm,neighbourhood,no.,borough,geometry.rings
0,4592631,Godorf,211,Rodenkirchen,"[[[6.994359341598065, 50.85835989408827], [6.9..."
1,3692013,Lövenich,308,Lindenthal,"[[[6.835101297207516, 50.957260493877406], [6...."
2,3654710,Weiden,307,Lindenthal,"[[[6.849501526802686, 50.94220412958467], [6.8..."
3,7376623,Junkersdorf,306,Lindenthal,"[[[6.854198181651118, 50.94052346246431], [6.8..."
4,5599212,Widdersdorf,309,Lindenthal,"[[[6.851763526592514, 50.97718504868292], [6.8..."


In [6]:
# check the dataframe and its datatypes

geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 5 columns):
sqm               86 non-null int64
neighbourhood     86 non-null object
no.               86 non-null object
borough           86 non-null object
geometry.rings    86 non-null object
dtypes: int64(1), object(4)
memory usage: 3.4+ KB


The column "no." is an object, but should be an integer. I'll adjust that.

In [7]:
geo['no.'] = geo['no.'].astype('int')
geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 5 columns):
sqm               86 non-null int64
neighbourhood     86 non-null object
no.               86 non-null int64
borough           86 non-null object
geometry.rings    86 non-null object
dtypes: int64(2), object(3)
memory usage: 3.4+ KB


**Getting the coordinates of the neighbourhoods: centroid of coordinates polygon**

The geo dataframe contains a polygon of coordinates for each neighbourhood. I will be using the centroid of each polygon as **the** coordinates of the neighbourhood.

For this step, I will be using the LinearRing constructor and cetroid method from the Shapely package.

However, the polygon in the geo dataframe is formatted as a nested list which cannot be processed by the LinearRing constructor. Therefore, I'll remove one nested list transform the column "geometry.rings".

In [8]:
# remove outer list of column

geo['geometry.rings'] = geo['geometry.rings'].map(lambda x: x[0])

In [9]:
# insert two new columns for latitude and longitude, made out of the centroid of the polygon per neighbourhood

geo.insert(len(geo.columns), 'latitude', geo['geometry.rings'].map(lambda coords: LinearRing(coords).centroid.y)) 
geo.insert(len(geo.columns), 'longitude', geo['geometry.rings'].map(lambda coords: LinearRing(coords).centroid.x))

# check dataframe
geo.head()

,sqm,neighbourhood,no.,borough,geometry.rings,latitude,longitude
0,4592631,Godorf,211,Rodenkirchen,"[[6.994359341598065, 50.85835989408827], [6.99...",50.853612,6.981604
1,3692013,Lövenich,308,Lindenthal,"[[6.835101297207516, 50.957260493877406], [6.8...",50.948982,6.830322
2,3654710,Weiden,307,Lindenthal,"[[6.849501526802686, 50.94220412958467], [6.84...",50.935889,6.829889
3,7376623,Junkersdorf,306,Lindenthal,"[[6.854198181651118, 50.94052346246431], [6.85...",50.923522,6.859912
4,5599212,Widdersdorf,309,Lindenthal,"[[6.851763526592514, 50.97718504868292], [6.85...",50.964576,6.843137


In [10]:
geo.shape

(86, 7)

The constructors of the Shapely library usually expect geographical coordinates in the order of: latitude, longitude.

However, the coordinates of the geometry ring are in the order: longitude, latitude.

Therefore, I'll swap them to be able to easily use the Shapely library for my analysis.

In [11]:
def swap_coord(coord):
    return [coord[1], coord[0]]

def swap_coords(coord_list):
    return list(map(swap_coord, coord_list))

geo['geometry.rings'] = geo['geometry.rings'].apply(swap_coords)

Since the neighbourhoods in Cologne have official neighbourhood numbers (see column "no."), I'm sorting the data according to the official neighbourhood number and reset the index.

In [12]:
# sort data by no.
geo = geo.sort_values(by = ['no.'])

# reset index
geo = geo.reset_index(drop = True)

# check data
geo.head()

,sqm,neighbourhood,no.,borough,geometry.rings,latitude,longitude
0,2361133,Altstadt/Süd,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",50.930223,6.953587
1,2863967,Neustadt/Süd,102,Innenstadt,"[[50.93748386396194, 6.9264717966942015], [50....",50.926103,6.946810
2,2455273,Altstadt/Nord,103,Innenstadt,"[[50.95008808829535, 6.9589532284153925], [50....",50.942170,6.954201
3,3489554,Neustadt/Nord,104,Innenstadt,"[[50.95928768013307, 6.9636858833452315], [50....",50.948038,6.948013
4,5239821,Deutz,105,Innenstadt,"[[50.94948568775351, 6.981916500484012], [50.9...",50.938282,6.980676


To get a first impression on Cologne and how the neighbourhoods are distributed, I'll have a look at the map:

In [13]:
# getting coordinates of Cologne
address = 'Cologne, Germany'
geolocator = Nominatim(user_agent = "cologne_explorer")
location_cologne = geolocator.geocode(address)
latitude_cologne = location_cologne.latitude
longitude_cologne = location_cologne.longitude

# create map of Cologne
map_cologne = folium.Map(location=[latitude_cologne, longitude_cologne], zoom_start = 10)

# add markers to map
for lat, lng, neighbourhood in zip(geo['latitude'], geo['longitude'], geo['neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'red',
        fill = True,
        fill_color = '#FF4500',
        fill_opacity = 0.5,
        parse_html = False).add_to(map_cologne)

# show map
map_cologne

##### 2. Socio-economic data for all neighbourhoods
To reflect the socio-economic state of each neighbourhood I will use the following data:
1. number of day nurseries in each neighbourhood
2. number of schools in each neighbourhood
3. number of registered cars in each neighbourhood
4. different indicators on households in each neighbourhood (e.g. single, couples, with children, without children etc.)
5. unemployment rates in each neighbourhood (e.g. rate of ALG1 or HARTZ4 receivers (that's two different kinds of unemployment money), female, male ones etc.)
6. inhabitants structure in each neighbourhood (e.g. rate of people per age group, per nationality etc.)

In [14]:
# request the geodata
# https://offenedaten-koeln.de/dataset/kindertagesst%C3%A4tten-k%C3%B6ln

nursery = json.loads(requests.get('https://geoportal.stadt-koeln.de/arcgis/rest/services/Stadtplanthemen/MapServer/9/query?geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&objectIds=&where=objectid%20is%20not%20null&time=&returnCountOnly=false&returnIdsOnly=false&returnGeometry=true&maxAllowableOffset=&outSR=4326&outFields=%2A&f=json').text)
#nursery

In [15]:
#Normalize semi-structured JSON data into a flat table on the features

nursery_data = json_normalize(nursery['features'])
nursery_data.head()

,attributes.ADRESSE,attributes.NR_STADTBEZIRK,attributes.NR_STADTTEIL,attributes.OBJECTID,attributes.POSTZUSTELLBEZIRK,attributes.STADTBEZIRK,attributes.STADTTEIL,attributes.TRAEGER_TYP,geometry.x,geometry.y
0,Immendorfer Hauptstraße 22,2,212,4865,50997,Rodenkirchen,Immendorf,städtisch,6.957222,50.859753
1,Peter-Röser-Straße 35,4,406,4866,50827,Ehrenfeld,Ossendorf,städtisch,6.913198,50.968268
2,Rather Schulstraße 18,8,808,4867,51107,Kalk,Rath/Heumar,städtisch,7.080394,50.920879
3,Rheinaustraße 3,1,101,4868,50676,Innenstadt,Altstadt/Süd,städtisch,6.960587,50.931854
4,Rheinsteinstraße 4,2,203,4869,50968,Rodenkirchen,Raderberg,städtisch,6.960326,50.906630


In [16]:
# clean data: rename columns accordingly and drop not needed columns

nursery_data.rename(columns = {"attributes.ADRESSE" : "day nurseries",
                               "attributes.STADTTEIL" : "neighbourhood"}, inplace = True)

nursery_data = nursery_data.drop(nursery_data.columns.difference(["day nurseries", "neighbourhood"]), axis = 1)

In [17]:
# how many day nurseries per neighbourhood

nursery_data = nursery_data.groupby('neighbourhood').count()

In [18]:
nursery_data.head()

,day nurseries
neighbourhood,
Altstadt/Nord,9
Altstadt/Süd,12
Bayenthal,7
Bickendorf,12
Bilderstöckchen,9


In [19]:
nursery_data.shape

(85, 1)

In [20]:
# merge back together

geo = geo.merge(nursery_data, on = 'neighbourhood')
geo.head()

,sqm,neighbourhood,no.,borough,geometry.rings,latitude,longitude,day nurseries
0,2361133,Altstadt/Süd,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",50.930223,6.953587,12
1,2863967,Neustadt/Süd,102,Innenstadt,"[[50.93748386396194, 6.9264717966942015], [50....",50.926103,6.946810,28
2,2455273,Altstadt/Nord,103,Innenstadt,"[[50.95008808829535, 6.9589532284153925], [50....",50.942170,6.954201,9
3,3489554,Neustadt/Nord,104,Innenstadt,"[[50.95928768013307, 6.9636858833452315], [50....",50.948038,6.948013,23
4,5239821,Deutz,105,Innenstadt,"[[50.94948568775351, 6.981916500484012], [50.9...",50.938282,6.980676,13


In [21]:
geo.shape

(85, 8)

In [22]:
# request the geodata
# https://offenedaten-koeln.de/dataset/schulen-k%C3%B6ln

schools = json.loads(requests.get('https://geoportal.stadt-koeln.de/arcgis/rest/services/Stadtplanthemen/MapServer/6/query?text=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&objectIds=&where=objectid+is+not+null&time=&returnCountOnly=false&returnIdsOnly=false&returnGeometry=true&maxAllowableOffset=&outSR=4326&outFields=*&f=json').text)
#schools


In [23]:
#Normalize semi-structured JSON data into a flat table on the features

school_data = json_normalize(schools['features'])
school_data.head()

,attributes.ADRESSE,attributes.NAME,attributes.NR_STADTBEZIRK,attributes.NR_STADTTEIL,attributes.OBJECTID,attributes.POSTZUSTELLBEZIRK,attributes.SCHULART,attributes.SCHULNAME,attributes.SCHULSTRASSE,attributes.SCHULTYP,attributes.STADTBEZIRK,attributes.STADTTEIL,attributes.TRAEGER,geometry.x,geometry.y
0,Lindenstr. 78,Berufskolleg an der Lindenstraße,1,102,4538,50674,Berufskolleg,None,Lindenstr. (BK),berufsbildend,Innenstadt,Neustadt/Süd,Stadt Köln,6.933669,50.932865
1,Neufelder Str. 32,Hilde-Domin-Schule,9,904,4539,51067,Schule für Kranke,None,Neufelder Str. (SK),allgemeinbildend,Mülheim,Holweide,Stadt Köln,7.055747,50.966257
2,Albert-Schweitzer-Str. 8,Otto-Lilienthal-Schule,7,711,4540,51147,Realschule,None,Albert-Schweitzer-Str. (RS),allgemeinbildend,Porz,Wahn,Stadt Köln,7.088293,50.864573
3,Humboldtstr. 81,Don-Bosco-Schule,7,706,4541,51145,Grundschule,None,Humboldtstr. (GGS),allgemeinbildend,Porz,Porz,Stadt Köln,7.070194,50.887473
4,Pfälzer Str. 30,Pfälzer Str.,1,102,4542,50677,Grundschule,None,Pfälzer Str. (GGS),allgemeinbildend,Innenstadt,Neustadt/Süd,Stadt Köln,6.943521,50.925908


In [24]:
# clean data: rename columns accordingly and drop not needed columns

school_data.rename(columns = {"attributes.NR_STADTTEIL" : "no.",
                               "attributes.SCHULART" : "school type"}, inplace = True)

school_data = school_data.drop(school_data.columns.difference(["no.", "school type"]), axis = 1)

In [25]:
school_data

,no.,school type
0,102,Berufskolleg
1,904,Schule für Kranke
2,711,Realschule
3,706,Grundschule
4,102,Grundschule
5,501,Grundschule
6,603,Grundschule
7,302,Förderschule
8,301,Grundschule
9,705,Hauptschule


In [26]:
# one hot encoding
school_data_onehot = pd.get_dummies(school_data[['school type']], prefix = "", prefix_sep = "")

# add column for neighbourhood no. back
school_data_onehot['no.'] = school_data['no.']

school_data_onehot.head()

,Berufskolleg,Freie Waldorfschule,Förderschule,Gesamtschule,Grundschule,Gymnasium,Hauptschule,Realschule,Schule für Kranke,Weiterbildungskolleg,no.
0,1,0,0,0,0,0,0,0,0,0,102
1,0,0,0,0,0,0,0,0,1,0,904
2,0,0,0,0,0,0,0,1,0,0,711
3,0,0,0,0,1,0,0,0,0,0,706
4,0,0,0,0,1,0,0,0,0,0,102


In [27]:
# group them and get the number of school types per neighbourhood

schools_grouped = school_data_onehot.groupby('no.').sum().reset_index()
schools_grouped.head()

,no.,Berufskolleg,Freie Waldorfschule,Förderschule,Gesamtschule,Grundschule,Gymnasium,Hauptschule,Realschule,Schule für Kranke,Weiterbildungskolleg
0,101,4,0,2,1,2,3,1,2,0,0
1,102,3,1,0,0,5,0,0,0,0,0
2,103,0,0,0,0,1,2,0,2,0,2
3,104,0,0,1,2,4,2,0,0,0,0
4,105,4,0,1,0,2,2,0,1,0,1


In [28]:
schools_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 11 columns):
no.                     80 non-null object
Berufskolleg            80 non-null uint8
Freie Waldorfschule     80 non-null uint8
Förderschule            80 non-null uint8
Gesamtschule            80 non-null uint8
Grundschule             80 non-null uint8
Gymnasium               80 non-null uint8
Hauptschule             80 non-null uint8
Realschule              80 non-null uint8
Schule für Kranke       80 non-null uint8
Weiterbildungskolleg    80 non-null uint8
dtypes: object(1), uint8(10)
memory usage: 1.5+ KB


In [29]:
# transform 'no.' into integer
schools_grouped['no.'] = schools_grouped['no.'].astype('int')
schools_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 11 columns):
no.                     80 non-null int64
Berufskolleg            80 non-null uint8
Freie Waldorfschule     80 non-null uint8
Förderschule            80 non-null uint8
Gesamtschule            80 non-null uint8
Grundschule             80 non-null uint8
Gymnasium               80 non-null uint8
Hauptschule             80 non-null uint8
Realschule              80 non-null uint8
Schule für Kranke       80 non-null uint8
Weiterbildungskolleg    80 non-null uint8
dtypes: int64(1), uint8(10)
memory usage: 1.5 KB


In [30]:
# merge back together

geo = geo.merge(schools_grouped, on = 'no.')
geo.head()

,sqm,neighbourhood,no.,borough,geometry.rings,latitude,longitude,day nurseries,Berufskolleg,Freie Waldorfschule,Förderschule,Gesamtschule,Grundschule,Gymnasium,Hauptschule,Realschule,Schule für Kranke,Weiterbildungskolleg
0,2361133,Altstadt/Süd,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",50.930223,6.953587,12,4,0,2,1,2,3,1,2,0,0
1,2863967,Neustadt/Süd,102,Innenstadt,"[[50.93748386396194, 6.9264717966942015], [50....",50.926103,6.946810,28,3,1,0,0,5,0,0,0,0,0
2,2455273,Altstadt/Nord,103,Innenstadt,"[[50.95008808829535, 6.9589532284153925], [50....",50.942170,6.954201,9,0,0,0,0,1,2,0,2,0,2
3,3489554,Neustadt/Nord,104,Innenstadt,"[[50.95928768013307, 6.9636858833452315], [50....",50.948038,6.948013,23,0,0,1,2,4,2,0,0,0,0
4,5239821,Deutz,105,Innenstadt,"[[50.94948568775351, 6.981916500484012], [50.9...",50.938282,6.980676,13,4,0,1,0,2,2,0,1,0,1


In [31]:
# request the geodata
# https://offenedaten-koeln.de/dataset/kfz-statistik-koeln

cars = json.loads(requests.get('https://geoportal.stadt-koeln.de/arcgis/rest/services/Statistische_Daten/QMFS_Kraftfahrzeuge/MapServer/1/query?where=objectid+is+not+null&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=4326&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&f=geojson').text)
#cars


In [32]:
#Normalize semi-structured JSON data into a flat table on the features

car_data = json_normalize(cars['features'])
car_data.head()

,geometry.coordinates,geometry.type,id,properties.KFZ_AA,properties.KFZ_AP,properties.KFZ_HA,properties.KFZ_HP,properties.NAME,properties.NUMMER,properties.OBJECTID,...,properties.PKW_AP,properties.PKW_HA,properties.PKW_HP,properties.PRIV_PKW_AA,properties.PRIV_PKW_AP,properties.PRIV_PKW_HA,properties.PRIV_PKW_HP,properties.SHAPE.AREA,properties.SHAPE.LEN,type
0,"[[[7.009698154969715, 50.98762355280711], [7.0...",Polygon,10,17580,49.592372,14146,40.719632,MÃ¼lheim,901,10,...,43.171881,12051,34.689119,11276,31.809078,10821,31.148532,7.065807e+06,15388.964696,Feature
1,"[[[6.890423336101029, 50.94649114299174], [6.8...",Polygon,11,7402,70.717493,6449,69.091493,Braunsfeld,304,11,...,60.666858,5484,58.752946,4650,44.425337,4065,43.550461,1.680550e+06,8676.896868,Feature
2,"[[[6.894022949910494, 50.99503466055852], [6.8...",Polygon,12,7732,92.921524,6381,84.304400,Ossendorf,406,12,...,73.008052,5138,67.882151,3749,45.054681,3376,44.602986,6.808229e+06,13733.120865,Feature
3,"[[[7.0547619255228255, 50.895118847617056], [7...",Polygon,1,6283,50.882734,5601,48.217975,Porz,706,1,...,45.602527,5010,43.130165,5392,43.666991,4786,41.201791,3.117291e+06,10466.991259,Feature
4,"[[[6.940278401301003, 50.936798424501795], [6....",Polygon,2,11771,46.191579,11434,45.664763,Altstadt/SÃ¼d,101,2,...,40.442648,10129,40.452893,7168,28.128556,7096,28.339790,2.361133e+06,8176.733201,Feature


In [33]:
# clean data: rename columns accordingly and drop not needed columns

car_data.rename(columns = {"properties.NUMMER" : "no.", "properties.KFZ_AP" : "mv rate", 
                           "properties.PKW_AP" : "car rate", "properties.PRIV_PKW_AP" : "p.car rate"}, inplace = True)

car_data = car_data.drop(car_data.columns.difference(["no.", "mv rate", "car rate", "p.car rate"]), axis=1)

car_data.head()

,mv rate,no.,car rate,p.car rate
0,49.592372,901,43.171881,31.809078
1,70.717493,304,60.666858,44.425337
2,92.921524,406,73.008052,45.054681
3,50.882734,706,45.602527,43.666991
4,46.191579,101,40.442648,28.128556


In [34]:
# transform 'no.' into integer
car_data['no.'] = car_data['no.'].astype('int')
car_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 4 columns):
mv rate       86 non-null float64
no.           86 non-null int64
car rate      86 non-null float64
p.car rate    86 non-null float64
dtypes: float64(3), int64(1)
memory usage: 2.8 KB


In [35]:
geo = geo.merge(car_data, on = 'no.')
geo.head()

,sqm,neighbourhood,no.,borough,geometry.rings,latitude,longitude,day nurseries,Berufskolleg,Freie Waldorfschule,...,Gesamtschule,Grundschule,Gymnasium,Hauptschule,Realschule,Schule für Kranke,Weiterbildungskolleg,mv rate,car rate,p.car rate
0,2361133,Altstadt/Süd,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",50.930223,6.953587,12,4,0,...,1,2,3,1,2,0,0,46.191579,40.442648,28.128556
1,2863967,Neustadt/Süd,102,Innenstadt,"[[50.93748386396194, 6.9264717966942015], [50....",50.926103,6.946810,28,3,1,...,0,5,0,0,0,0,0,41.481053,35.666763,29.953717
2,2455273,Altstadt/Nord,103,Innenstadt,"[[50.95008808829535, 6.9589532284153925], [50....",50.942170,6.954201,9,0,0,...,0,1,2,0,2,0,2,68.354804,59.608793,29.216405
3,3489554,Neustadt/Nord,104,Innenstadt,"[[50.95928768013307, 6.9636858833452315], [50....",50.948038,6.948013,23,0,0,...,2,4,2,0,0,0,0,59.043342,50.140570,34.666146
4,5239821,Deutz,105,Innenstadt,"[[50.94948568775351, 6.981916500484012], [50.9...",50.938282,6.980676,13,4,0,...,0,2,2,0,1,0,1,69.130062,58.119903,31.678352


In [36]:
geo.shape

(79, 21)

In [37]:
# request the geodata
# https://offenedaten-koeln.de/dataset/haushalte-statistik-koeln

households = json.loads(requests.get('https://geoportal.stadt-koeln.de/arcgis/rest/services/Statistische_Daten/QMFS_Haushalte/MapServer/1/query?where=objectid+is+not+null&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=4326&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&f=geojson').text)
#households

In [38]:
#Normalize semi-structured JSON data into a flat table on the features

household_data = json_normalize(households['features'])
#household_data

In [39]:
# clean data: rename columns accordingly and drop not needed columns

household_data.rename(columns = {"properties.NUMMER" : "no.", "properties.HH_HG_1PERS_AP" : "1 person hh per",
                                "properties.HH_HG_2PERS_AP" : "2 person hh per", "properties.HH_HG_3PERS_AP" : "3 person hh per", 
                                "properties.HH_HG_4PERS_AP" : "4 person hh per", "properties.HH_HG_AB5PERS_AP" : "mt5 person hh per", 
                                "properties.HH_HT_SINGLE_AP" : "single hh per", "properties.HH_HT_EHEP_O_KIND_AP" : "mcouple no child hh per",
                                "properties.HH_HT_EHEP_M_KIND_AP" : "mcouple child hh per",
                                "properties.HH_HT_PAAR_O_KIND_AP" : "couple no child hh per",
                                "properties.HH_HT_PAAR_M_KIND_AP" : "couple child hh per",
                                "properties.HH_HT_ALLEINERZ_AP" : "single parent hh per", 
                                "properties.HH_HT_SMPHH_O_K_AP" : "other no child hh per"}, inplace = True)

household_data = household_data.drop(household_data.columns.difference(["no.", "1 person hh per", "2 person hh per",
                                                                        "3 person hh per", "4 person hh per", 
                                                                        "mt5 person hh per", "single hh per", 
                                                                        "mcouple no child hh per", "mcouple child hh per",
                                                                        "couple no child hh per", "couple child hh per",
                                                                        "single parent hh per", "other no child hh per"]), axis=1)
        

In [40]:
household_data.head()

,1 person hh per,2 person hh per,3 person hh per,4 person hh per,mt5 person hh per,single parent hh per,mcouple child hh per,mcouple no child hh per,couple child hh per,couple no child hh per,single hh per,other no child hh per,no.
0,40.7,32.0,14.2,9.0,4.2,5.3,14.4,28.2,2.2,3.4,40.7,5.8,908
1,54.3,24.4,10.3,7.2,3.8,4.9,11.1,16.7,2.4,5.1,54.3,5.5,901
2,56.0,26.5,9.5,6.1,1.9,2.7,10.0,17.7,2.2,7.3,56.0,4.0,304
3,38.1,27.9,15.6,12.7,5.8,7.3,17.9,23.6,3.0,4.1,38.1,6.0,406
4,44.0,29.3,12.6,9.5,4.6,4.9,13.6,25.2,2.1,4.0,44.0,6.2,706


In [41]:
# transform 'no.' into integer
household_data['no.'] = household_data['no.'].astype('int')
household_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 13 columns):
1 person hh per            86 non-null float64
2 person hh per            86 non-null float64
3 person hh per            86 non-null float64
4 person hh per            86 non-null float64
mt5 person hh per          86 non-null float64
single parent hh per       86 non-null float64
mcouple child hh per       86 non-null float64
mcouple no child hh per    86 non-null float64
couple child hh per        86 non-null float64
couple no child hh per     86 non-null float64
single hh per              86 non-null float64
other no child hh per      86 non-null float64
no.                        86 non-null int64
dtypes: float64(12), int64(1)
memory usage: 8.8 KB


In [42]:
# merge back together

geo = geo.merge(household_data, on = 'no.')
geo.head()

,sqm,neighbourhood,no.,borough,geometry.rings,latitude,longitude,day nurseries,Berufskolleg,Freie Waldorfschule,...,3 person hh per,4 person hh per,mt5 person hh per,single parent hh per,mcouple child hh per,mcouple no child hh per,couple child hh per,couple no child hh per,single hh per,other no child hh per
0,2361133,Altstadt/Süd,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",50.930223,6.953587,12,4,0,...,5.7,3.1,1.1,1.9,4.8,12.3,1.2,6.0,69.5,4.2
1,2863967,Neustadt/Süd,102,Innenstadt,"[[50.93748386396194, 6.9264717966942015], [50....",50.926103,6.946810,28,3,1,...,6.8,4.1,1.2,2.5,6.2,10.7,2.1,7.7,66.8,4.0
2,2455273,Altstadt/Nord,103,Innenstadt,"[[50.95008808829535, 6.9589532284153925], [50....",50.942170,6.954201,9,0,0,...,5.4,2.6,0.9,1.6,4.4,11.9,1.2,6.2,70.6,4.2
3,3489554,Neustadt/Nord,104,Innenstadt,"[[50.95928768013307, 6.9636858833452315], [50....",50.948038,6.948013,23,0,0,...,7.3,4.6,1.0,2.1,6.9,13.4,1.9,6.6,65.3,3.7
4,5239821,Deutz,105,Innenstadt,"[[50.94948568775351, 6.981916500484012], [50.9...",50.938282,6.980676,13,4,0,...,8.1,4.4,1.4,2.9,7.1,16.2,1.7,7.0,60.2,4.9


In [43]:
# request the geodata
# https://offenedaten-koeln.de/dataset/arbeitsmarkt-statistik-koeln

unemployment = json.loads(requests.get('https://geoportal.stadt-koeln.de/arcgis/rest/services/Statistische_Daten/QMFS_Arbeitsmarkt/MapServer/1/query?where=objectid+is+not+null&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=4326&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&f=geojson').text)
#unemployment

In [44]:
#Normalize semi-structured JSON data into a flat table on the features

unemployment_data = json_normalize(unemployment['features'])
unemployment_data.head()

,geometry.coordinates,geometry.type,id,properties.AM_SGB2_DEUTSCH_AA,properties.AM_SGB2_DEUTSCH_AP,properties.AM_SGB2_DEUTSCH_HA,properties.AM_SGB2_DEUTSCH_HP,properties.AM_SGB2_FRAU_AA,properties.AM_SGB2_FRAU_AP,properties.AM_SGB2_FRAU_HA,...,properties.AM_SGB_UNTER25_AA,properties.AM_SGB_UNTER25_AP,properties.AM_SGB_UNTER25_HA,properties.AM_SGB_UNTER25_HP,properties.NAME,properties.NUMMER,properties.OBJECTID,properties.SHAPE.AREA,properties.SHAPE.LEN,type
0,"[[[7.010640423873277, 50.99586620162058], [7.0...",Polygon,9,192,5.793470,223,7.7,141,7.543047,169,...,28,7.267593,36,9.6,Stammheim,908,9,3.741682e+06,12152.731636,Feature
1,"[[[7.009698154969715, 50.98762355280711], [7.0...",Polygon,10,1220,7.502731,1646,11.2,1112,10.254232,1215,...,208,8.698397,252,11.2,MÃ¼lheim,901,10,7.065807e+06,15388.964696,Feature
2,"[[[6.890423336101029, 50.94649114299174], [6.8...",Polygon,11,82,1.432750,126,2.6,47,1.520107,58,...,8,1.399799,15,3.2,Braunsfeld,304,11,1.680550e+06,8676.896868,Feature
3,"[[[6.894022949910494, 50.99503466055852], [6.8...",Polygon,12,231,5.070323,313,7.7,204,7.494347,260,...,34,5.256170,41,7.4,Ossendorf,406,12,6.808229e+06,13733.120865,Feature
4,"[[[7.0547619255228255, 50.895118847617056], [7...",Polygon,1,344,5.650124,423,7.7,311,8.303875,357,...,47,5.914090,79,11.4,Porz,706,1,3.117291e+06,10466.991259,Feature


In [45]:
# clean data: rename columns accordingly and drop not needed columns

unemployment_data.rename(columns = {"properties.NUMMER" : "no.", 
                                    "properties.AM_SGB2_INSG_AP" : "ur alg1 all",
                                    "properties.AM_SGB2_MANN_AP" : "ur alg1 men", 
                                    "properties.AM_SGB2_FRAU_AP" : "ur alg1 women",
                                    "properties.AM_SGB2_DEUTSCH_AP" : "ur alg1 germans",
                                    "properties.AM_SGB2_UNTER25_AP" : "ur alg1 below 25", 
                                    "properties.AM_SGB3_INSG_AP" : "ur hartz41 all",
                                    "properties.AM_SGB3_MANN_AP" : "ur hartz4 men",
                                    "properties.AM_SGB3_FRAU_AP" : "ur hartz4 women",
                                    "properties.AM_SGB3_DEUTSCH_AP" : "ur hartz4 germans",
                                    "properties.AM_SGB3_UNTER25_AP" : "ur hartz4 below 25",}, inplace = True)

unemployment_data = unemployment_data.drop(unemployment_data.columns.difference(["no.", "ur alg1 all", "ur alg1 men",
                                                                                 "ur alg1 women", "ur alg1 germans", 
                                                                                 "ur alg1 below 25", "ur hartz41 all",
                                                                                 "ur hartz4 men", "ur hartz4 women",
                                                                                 "ur hartz4 germans",
                                                                                 "ur hartz4 below 25"]), axis=1)

unemployment_data.head()


,ur alg1 germans,ur alg1 women,ur alg1 all,ur alg1 men,ur alg1 below 25,ur hartz4 germans,ur hartz4 women,ur hartz41 all,ur hartz4 men,ur hartz4 below 25,no.
0,5.793470,7.543047,7.502672,7.468912,3.893354,1.719936,1.551407,2.005923,2.438828,3.374240,908
1,7.502731,10.254232,10.669043,10.964672,6.607436,1.974079,1.982608,2.484687,2.890686,2.090961,901
2,1.432750,1.520107,2.003295,2.483199,0.699900,1.590003,2.037590,1.762899,1.496287,0.699900,304
3,5.070323,7.494347,6.540983,5.670767,2.782678,1.755956,1.616428,1.988459,2.321367,2.473492,406
4,5.650124,8.303875,7.683664,7.101177,3.774951,1.954549,1.762237,2.099079,2.417422,2.139139,706


In [46]:
# transform 'no.' into integer
unemployment_data['no.'] = unemployment_data['no.'].astype('int')
unemployment_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 11 columns):
ur alg1 germans       86 non-null float64
ur alg1 women         86 non-null float64
ur alg1 all           86 non-null float64
ur alg1 men           86 non-null float64
ur alg1 below 25      86 non-null float64
ur hartz4 germans     86 non-null float64
ur hartz4 women       86 non-null float64
ur hartz41 all        86 non-null float64
ur hartz4 men         86 non-null float64
ur hartz4 below 25    86 non-null float64
no.                   86 non-null int64
dtypes: float64(10), int64(1)
memory usage: 7.5 KB


In [47]:
# merge together

geo = geo.merge(unemployment_data, on = 'no.')
geo.head()

,sqm,neighbourhood,no.,borough,geometry.rings,latitude,longitude,day nurseries,Berufskolleg,Freie Waldorfschule,...,ur alg1 germans,ur alg1 women,ur alg1 all,ur alg1 men,ur alg1 below 25,ur hartz4 germans,ur hartz4 women,ur hartz41 all,ur hartz4 men,ur hartz4 below 25
0,2361133,Altstadt/Süd,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",50.930223,6.953587,12,4,0,...,3.270990,3.211575,4.238441,5.126187,2.808915,2.149301,2.220348,2.255147,2.279591,1.053343
1,2863967,Neustadt/Süd,102,Innenstadt,"[[50.93748386396194, 6.9264717966942015], [50....",50.926103,6.946810,28,3,1,...,2.436985,2.187418,3.139752,4.004590,1.372338,2.084764,2.276337,2.219041,2.164424,0.640424
2,2455273,Altstadt/Nord,103,Innenstadt,"[[50.95008808829535, 6.9589532284153925], [50....",50.942170,6.954201,9,0,0,...,1.937897,1.899198,2.847161,3.567507,2.463683,1.949501,2.191383,2.094041,2.003668,1.285400
3,3489554,Neustadt/Nord,104,Innenstadt,"[[50.95928768013307, 6.9636858833452315], [50....",50.948038,6.948013,23,0,0,...,1.701992,1.760598,2.146948,2.491517,1.085149,2.070533,2.427110,2.230441,2.049839,1.168622
4,5239821,Deutz,105,Innenstadt,"[[50.94948568775351, 6.981916500484012], [50.9...",50.938282,6.980676,13,4,0,...,3.399355,3.205978,4.172247,5.009600,2.315082,1.445703,1.408687,1.614330,1.790666,1.041787


In [48]:
# request the geodata
# https://offenedaten-koeln.de/dataset/einwohner-statistik-koeln

inhabitants = json.loads(requests.get('https://geoportal.stadt-koeln.de/arcgis/rest/services/Statistische_Daten/QMFS_Einwohner/MapServer/4/query?where=objectid+is+not+null&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=4326&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&resultOffset=&resultRecordCount=&f=geojson').text)
#inhabitants


In [49]:
#Normalize semi-structured JSON data into a flat table on the features

inhabitants_data = json_normalize(inhabitants['features'])
inhabitants_data.head()

,geometry.coordinates,geometry.type,id,properties.AUSL1GEN_AA,properties.AUSL1GEN_AP,properties.AUSL1GEN_HA,properties.AUSL1GEN_HP,properties.AUSL23GEN_AA,properties.AUSL23GEN_AP,properties.AUSL23GEN_HA,...,properties.WEIBLICH_HP,properties.ZUZUG_AA,properties.ZUZUG_AP,properties.ZUZUG_BINNEN_AA,properties.ZUZUG_BINNEN_AP,properties.ZUZUG_BINNEN_HA,properties.ZUZUG_BINNEN_HP,properties.ZUZUG_HA,properties.ZUZUG_HP,type
0,"[[[6.909529933340396, 50.96003050116499], [6.9...",Polygon,7,6094,45.5,5853,47.5,1468,11.0,1810,...,49.615751,2893,7.6,3075,8.1,3348,9.4,2874,8.1,Feature
1,"[[[7.065034590248008, 50.90002220335997], [7.0...",Polygon,8,2626,45.0,1827,36.8,676,11.6,511,...,52.020437,378,5.4,510,7.3,432,6.7,276,4.3,Feature
2,"[[[7.010640423873277, 50.99586620162058], [7.0...",Polygon,9,978,28.3,752,26.9,351,10.2,296,...,52.283507,241,3.0,449,5.6,554,7.5,241,3.3,Feature
3,"[[[7.009698154969715, 50.98762355280711], [7.0...",Polygon,10,10943,48.0,9496,47.3,3010,13.2,3118,...,48.557565,2589,6.1,2958,6.9,3682,8.9,3304,8.0,Feature
4,"[[[7.0547619255228255, 50.895118847617056], [7...",Polygon,1,2665,33.7,2070,31.1,734,9.3,681,...,52.198156,614,4.1,846,5.7,1234,8.8,708,5.1,Feature


In [50]:
# clean data: rename columns accordingly and drop not needed columns
# very broad table, rename the only two columns I will need which are not a rate first:

inhabitants_data.rename(columns = {"properties.NUMMER" : "no.", "properties.EWD_AA" : "inhabitants",
                                  "properties.A_U_18_AA" : "inhabitants younger 18"}, inplace = True)

# drop all columns that do not contain a rate:

inhabitants_data = inhabitants_data.drop(inhabitants_data.filter(regex='_AA').columns, axis=1)
inhabitants_data = inhabitants_data.drop(inhabitants_data.filter(regex='_HA').columns, axis=1)
inhabitants_data = inhabitants_data.drop(inhabitants_data.filter(regex='_HP').columns, axis=1)
inhabitants_data.drop(['type', 'geometry.coordinates', 'geometry.type', 'id'], 1, inplace=True)

# remove "properties" from column name

inhabitants_data.columns = inhabitants_data.columns.str.replace('properties.', '')

In [51]:
inhabitants_data.head()

,AUSL1GEN_AP,AUSL23GEN_AP,AUSLAENDER_AP,AUSLAND_AP,AUSSIEDLER_AP,A_18_30_AP,A_30_65_AP,A_65_80_AP,A_AB_80_AP,inhabitants younger 18,...,WD_0_2_AP,WD_10_15_AP,WD_15_20_AP,WD_20_30_AP,WD_2_5_AP,WD_5_10_AP,WD_AB30_AP,WEIBLICH_AP,ZUZUG_AP,ZUZUG_BINNEN_AP
0,45.5,11.0,20.0,23.6,7.7,22.0,54.5,8.6,2.9,4556,...,27.4,11.6,6.8,5.5,22.9,19.3,6.5,50.4,7.6,8.1
1,45.0,11.6,47.3,60.9,14.8,16.4,38.9,13.8,6.4,1712,...,21.8,10.0,9.8,8.0,21.7,20.1,8.7,51.0,5.4,7.3
2,28.3,10.2,16.6,27.6,26.8,14.3,45.8,15.9,6.6,1390,...,17.0,11.6,7.7,8.3,22.0,19.4,13.9,51.5,3.0,5.6
3,48.0,13.2,32.6,35.1,6.5,19.1,49.7,10.3,3.8,7337,...,23.3,11.6,7.5,7.2,21.5,21.1,7.8,48.7,6.1,6.9
4,33.7,9.3,22.8,35.4,15.6,14.8,48.5,13.3,6.2,2567,...,19.9,12.4,8.1,7.2,20.2,24.5,7.8,51.9,4.1,5.7


In [52]:
# transform 'no.' into integer
inhabitants_data['no.'] = inhabitants_data['no.'].astype('int')
inhabitants_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 63 columns):
AUSL1GEN_AP               86 non-null float64
AUSL23GEN_AP              86 non-null float64
AUSLAENDER_AP             86 non-null float64
AUSLAND_AP                86 non-null float64
AUSSIEDLER_AP             86 non-null float64
A_18_30_AP                86 non-null float64
A_30_65_AP                86 non-null float64
A_65_80_AP                86 non-null float64
A_AB_80_AP                86 non-null float64
inhabitants younger 18    86 non-null int64
A_U_18_AP                 86 non-null float64
BALKANGRUPPE_AP           86 non-null float64
BULGARIEN_AP              86 non-null float64
DEUTSCHE_AP               86 non-null float64
DEUTSCHE_MHG_AP           86 non-null float64
DEUTSCHLAND_AP            86 non-null float64
EHEMUDSSR_AP              86 non-null float64
EINGEBUERGERTE_AP         86 non-null float64
EVANGELISCH_AP            86 non-null float64
inhabitants              

In [53]:
# merge together

geo = geo.merge(inhabitants_data, on = 'no.')
geo.head()

,sqm,neighbourhood,no.,borough,geometry.rings,latitude,longitude,day nurseries,Berufskolleg,Freie Waldorfschule,...,WD_0_2_AP,WD_10_15_AP,WD_15_20_AP,WD_20_30_AP,WD_2_5_AP,WD_5_10_AP,WD_AB30_AP,WEIBLICH_AP,ZUZUG_AP,ZUZUG_BINNEN_AP
0,2361133,Altstadt/Süd,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",50.930223,6.953587,12,4,0,...,28.0,9.9,6.5,6.9,21.0,18.4,9.2,49.9,8.6,8.6
1,2863967,Neustadt/Süd,102,Innenstadt,"[[50.93748386396194, 6.9264717966942015], [50....",50.926103,6.946810,28,3,1,...,28.4,10.2,5.8,6.5,22.9,18.5,7.7,50.2,8.9,7.8
2,2455273,Altstadt/Nord,103,Innenstadt,"[[50.95008808829535, 6.9589532284153925], [50....",50.942170,6.954201,9,0,0,...,30.2,9.4,6.1,6.4,23.2,18.0,6.7,47.3,10.2,8.6
3,3489554,Neustadt/Nord,104,Innenstadt,"[[50.95928768013307, 6.9636858833452315], [50....",50.948038,6.948013,23,0,0,...,25.8,11.1,6.8,7.1,21.2,19.0,9.0,49.8,7.7,6.7
4,5239821,Deutz,105,Innenstadt,"[[50.94948568775351, 6.981916500484012], [50.9...",50.938282,6.980676,13,4,0,...,27.0,11.0,7.3,7.2,18.9,18.0,10.6,50.4,8.5,7.2


For some indicators, I still have absolute values that do not possess a lot of explanatory power (e.g. number of day nurseries or schools for each neighbourhood). Therefore, I'll put them in relation (as far as possible, I don't have all the data I would need, e.g. number of kids younger than 3 years).

In [54]:
# number of day nurseries per inhabitants

geo.insert(5, 'day nurseries pp', geo['day nurseries']/geo['inhabitants'])

geo.drop(['day nurseries'], 1, inplace = True)

# number of non-professional schools per inhabitant younger than 18

geo.insert(len(geo.columns), 'Freie Waldorfschule pp', geo['Freie Waldorfschule']/geo['inhabitants younger 18'])
geo.insert(len(geo.columns), 'Förderschule pp', geo['Förderschule']/geo['inhabitants younger 18'])
geo.insert(len(geo.columns), 'Gesamtschule pp', geo['Gesamtschule']/geo['inhabitants younger 18'])
geo.insert(len(geo.columns), 'Grundschule pp', geo['Grundschule']/geo['inhabitants younger 18'])
geo.insert(len(geo.columns), 'Gymnasium pp', geo['Gymnasium']/geo['inhabitants younger 18'])
geo.insert(len(geo.columns), 'Hauptschule pp', geo['Hauptschule']/geo['inhabitants younger 18'])
geo.insert(len(geo.columns), 'Realschule pp', geo['Realschule']/geo['inhabitants younger 18'])
geo.insert(len(geo.columns), 'Schule für Kranke pp', geo['Schule für Kranke']/geo['inhabitants younger 18'])

# number of professional schools per inhabitant

geo.insert(len(geo.columns), 'Berufskolleg pp', geo['Berufskolleg']/geo['inhabitants'])
geo.insert(len(geo.columns), 'Weiterbildungskolleg pp', geo['Weiterbildungskolleg']/geo['inhabitants'])

# drop old columns

geo.drop(['Freie Waldorfschule', 'Förderschule', 'Gesamtschule', 'Grundschule', 'Gymnasium', 'Hauptschule',
          'Realschule', 'Schule für Kranke', 'Berufskolleg', 'Weiterbildungskolleg', 'inhabitants younger 18'], 1, inplace = True)

# population density

geo.insert(5, 'inhabitants/sqm', geo['sqm']/geo['inhabitants'])

# drop old columns

geo.drop(['sqm', 'inhabitants'], 1, inplace = True)


In [55]:
# sorting according to official neighbourhood numbers:

# sort data by no.
geo=geo.sort_values(by=['no.'])

# reset index
geo=geo.reset_index(drop=True)

# check data
geo.head()

,neighbourhood,no.,borough,geometry.rings,inhabitants/sqm,day nurseries pp,latitude,longitude,mv rate,car rate,...,Freie Waldorfschule pp,Förderschule pp,Gesamtschule pp,Grundschule pp,Gymnasium pp,Hauptschule pp,Realschule pp,Schule für Kranke pp,Berufskolleg pp,Weiterbildungskolleg pp
0,Altstadt/Süd,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",84.698246,0.000430,50.930223,6.953587,46.191579,40.442648,...,0.000000,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.000000
1,Neustadt/Süd,102,Innenstadt,"[[50.93748386396194, 6.9264717966942015], [50....",74.182583,0.000725,50.926103,6.946810,41.481053,35.666763,...,0.000248,0.000000,0.000000,0.001239,0.000000,0.000000,0.000000,0.0,0.000078,0.000000
2,Altstadt/Nord,103,Innenstadt,"[[50.95008808829535, 6.9589532284153925], [50....",134.388232,0.000493,50.942170,6.954201,68.354804,59.608793,...,0.000000,0.000000,0.000000,0.000742,0.001484,0.000000,0.001484,0.0,0.000000,0.000109
3,Neustadt/Nord,104,Innenstadt,"[[50.95928768013307, 6.9636858833452315], [50....",121.969731,0.000804,50.948038,6.948013,59.043342,50.140570,...,0.000000,0.000333,0.000667,0.001333,0.000667,0.000000,0.000000,0.0,0.000000,0.000000
4,Deutz,105,Innenstadt,"[[50.94948568775351, 6.981916500484012], [50.9...",332.813834,0.000826,50.938282,6.980676,69.130062,58.119903,...,0.000000,0.000566,0.000000,0.001133,0.001133,0.000000,0.000566,0.0,0.000254,0.000064


In [56]:
# check if only rates are left in columns

list(geo)

['neighbourhood',
 'no.',
 'borough',
 'geometry.rings',
 'inhabitants/sqm',
 'day nurseries pp',
 'latitude',
 'longitude',
 'mv rate',
 'car rate',
 'p.car rate',
 '1 person hh per',
 '2 person hh per',
 '3 person hh per',
 '4 person hh per',
 'mt5 person hh per',
 'single parent hh per',
 'mcouple child hh per',
 'mcouple no child hh per',
 'couple child hh per',
 'couple no child hh per',
 'single hh per',
 'other no child hh per',
 'ur alg1 germans',
 'ur alg1 women',
 'ur alg1 all',
 'ur alg1 men',
 'ur alg1 below 25',
 'ur hartz4 germans',
 'ur hartz4 women',
 'ur hartz41 all',
 'ur hartz4 men',
 'ur hartz4 below 25',
 'AUSL1GEN_AP',
 'AUSL23GEN_AP',
 'AUSLAENDER_AP',
 'AUSLAND_AP',
 'AUSSIEDLER_AP',
 'A_18_30_AP',
 'A_30_65_AP',
 'A_65_80_AP',
 'A_AB_80_AP',
 'A_U_18_AP',
 'BALKANGRUPPE_AP',
 'BULGARIEN_AP',
 'DEUTSCHE_AP',
 'DEUTSCHE_MHG_AP',
 'DEUTSCHLAND_AP',
 'EHEMUDSSR_AP',
 'EINGEBUERGERTE_AP',
 'EVANGELISCH_AP',
 'EWD_AP',
 'FORTZUG_AP',
 'FORTZUG_BINNEN_AP',
 'GESCHIEDE

In [57]:
# drop columns that are not needed

geo.drop(['NAME', 'SHAPE.AREA', 'SHAPE.LEN', 'OBJECTID'], 1, inplace = True)

list(geo)

['neighbourhood',
 'no.',
 'borough',
 'geometry.rings',
 'inhabitants/sqm',
 'day nurseries pp',
 'latitude',
 'longitude',
 'mv rate',
 'car rate',
 'p.car rate',
 '1 person hh per',
 '2 person hh per',
 '3 person hh per',
 '4 person hh per',
 'mt5 person hh per',
 'single parent hh per',
 'mcouple child hh per',
 'mcouple no child hh per',
 'couple child hh per',
 'couple no child hh per',
 'single hh per',
 'other no child hh per',
 'ur alg1 germans',
 'ur alg1 women',
 'ur alg1 all',
 'ur alg1 men',
 'ur alg1 below 25',
 'ur hartz4 germans',
 'ur hartz4 women',
 'ur hartz41 all',
 'ur hartz4 men',
 'ur hartz4 below 25',
 'AUSL1GEN_AP',
 'AUSL23GEN_AP',
 'AUSLAENDER_AP',
 'AUSLAND_AP',
 'AUSSIEDLER_AP',
 'A_18_30_AP',
 'A_30_65_AP',
 'A_65_80_AP',
 'A_AB_80_AP',
 'A_U_18_AP',
 'BALKANGRUPPE_AP',
 'BULGARIEN_AP',
 'DEUTSCHE_AP',
 'DEUTSCHE_MHG_AP',
 'DEUTSCHLAND_AP',
 'EHEMUDSSR_AP',
 'EINGEBUERGERTE_AP',
 'EVANGELISCH_AP',
 'EWD_AP',
 'FORTZUG_AP',
 'FORTZUG_BINNEN_AP',
 'GESCHIEDE

In [58]:
geo.head()

,neighbourhood,no.,borough,geometry.rings,inhabitants/sqm,day nurseries pp,latitude,longitude,mv rate,car rate,...,Freie Waldorfschule pp,Förderschule pp,Gesamtschule pp,Grundschule pp,Gymnasium pp,Hauptschule pp,Realschule pp,Schule für Kranke pp,Berufskolleg pp,Weiterbildungskolleg pp
0,Altstadt/Süd,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",84.698246,0.000430,50.930223,6.953587,46.191579,40.442648,...,0.000000,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.000000
1,Neustadt/Süd,102,Innenstadt,"[[50.93748386396194, 6.9264717966942015], [50....",74.182583,0.000725,50.926103,6.946810,41.481053,35.666763,...,0.000248,0.000000,0.000000,0.001239,0.000000,0.000000,0.000000,0.0,0.000078,0.000000
2,Altstadt/Nord,103,Innenstadt,"[[50.95008808829535, 6.9589532284153925], [50....",134.388232,0.000493,50.942170,6.954201,68.354804,59.608793,...,0.000000,0.000000,0.000000,0.000742,0.001484,0.000000,0.001484,0.0,0.000000,0.000109
3,Neustadt/Nord,104,Innenstadt,"[[50.95928768013307, 6.9636858833452315], [50....",121.969731,0.000804,50.948038,6.948013,59.043342,50.140570,...,0.000000,0.000333,0.000667,0.001333,0.000667,0.000000,0.000000,0.0,0.000000,0.000000
4,Deutz,105,Innenstadt,"[[50.94948568775351, 6.981916500484012], [50.9...",332.813834,0.000826,50.938282,6.980676,69.130062,58.119903,...,0.000000,0.000566,0.000000,0.001133,0.001133,0.000000,0.000566,0.0,0.000254,0.000064


In [59]:
geo.shape

(79, 99)

**This is the final socio-economic data set for the analysis and comparison of Cologne's neighbourhoods.**

##### 3. Foursquare venues data for all neighbourhoods

Foursquare is a location data platform that gives you venues in different catagories, their location and also rating and tips from users who have been there.

For extracting data from Foursquare, we need:
* Foursquare credentials
* a function that extracts the category of the venue from the api request results

In [60]:
# @hidden_cell
# foursquare credentials:

CLIENT_ID = '4HUEVXA0O42MVG40QDMA3REVVMDJRVUDNMHFRLXSAOMSZTCQ' # your Foursquare ID
CLIENT_SECRET = '45XVQYP1ULVF3T2Z1LDDR2CPM0N5LX03S3E1RJBNHWGZLXA1' # your Foursquare Secret
VERSION = '20190513' # Foursquare API version


In [61]:
# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

To see how the Foursquare data look like, I'll map the venues of two neighbourhoods onto the map of Cologne.

In [62]:
# explore top 300 venues within radius of 750 m in Altstadt-Süd

LIMIT = 300 # limit of number of venues returned by Foursquare API
radius = 750 # define radius
neighborhood_latitude=geo.loc[0, 'latitude'] # Altsadt-Süd
neighborhood_longitude=geo.loc[0, 'longitude'] # Altstadt-Süd

 # create URL
url_altstadtsüd = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude,
    neighborhood_longitude, 
    radius, 
    LIMIT)

venues_altstadtsüd = requests.get(url_altstadtsüd).json()

# clean json and structure it in pandas dataframe

venues = venues_altstadtsüd['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Motel One Köln-Waidmarkt,Hotel,50.931557,6.954513
1,Holiday Inn Express,Hotel,50.930371,6.953661
2,Hotel im Wasserturm,Hotel,50.931745,6.951255
3,Motel One Lounge,Lounge,50.931571,6.954438
4,AgrippaFit,Gym / Fitness Center,50.933092,6.951967


In [63]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

86 venues were returned by Foursquare.


In [64]:
# create map of Cologne
map_cologne = folium.Map(location=[latitude_cologne, longitude_cologne], zoom_start = 13)

# add markers of venues to map
for lat, lng, name in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_cologne)
    
# Add the borders of Altstadt Süd to the map

altstadtsüd_poly = folium.PolyLine(geo.loc[0, 'geometry.rings'], color="red", weight=2.5, opacity=1)
altstadtsüd_poly.add_to(map_cologne)                                   

# show map
map_cologne

As can be seen on the map, the problem with requesting data from Foursquare is, that it's only possible within the radius of a certain point. Unfortunately, the neighbourhoods of Cologne don't have the shape of a circle. This leads either to venues within the neighbourhood missing or not belonging to the neighbourhood.

Therefore, I'll extend the radius of the Foursquare request up to 2km to make sure I'll cover any venues within the borders of each neighbourhood.

In a second step, I'll then drop those venues per neigbhbourhood which are outside the according neighbourhood.

Now, I'll consider all neighbourhoods of Cologne.

In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighbourhood', 
                  'neighborhood latitude', 
                  'neighborhood longitude', 
                  'venue', 
                  'venue latitude', 
                  'venue longitude', 
                  'venue category']
    
    return(nearby_venues)

Now reuse the code to run the above function on each neighborhood and create a new dataframe called cologne_venues.

In [66]:
cologne_venues = getNearbyVenues(names=geo['neighbourhood'],
                                   latitudes=geo['latitude'],
                                   longitudes=geo['longitude']
                                  )

Altstadt/Süd
Neustadt/Süd
Altstadt/Nord
Neustadt/Nord
Deutz
Bayenthal
Marienburg
Raderberg
Raderthal
Zollstock
Rondorf
Rodenkirchen
Weiß
Sürth
Godorf
Meschenich
Klettenberg
Sülz
Lindenthal
Braunsfeld
Müngersdorf
Junkersdorf
Weiden
Lövenich
Widdersdorf
Ehrenfeld
Neuehrenfeld
Bickendorf
Vogelsang
Bocklemünd/Mengenich
Ossendorf
Nippes
Mauenheim
Riehl
Niehl
Weidenpesch
Longerich
Bilderstöckchen
Merkenich
Seeberg
Heimersdorf
Lindweiler
Pesch
Esch/Auweiler
Volkhoven/Weiler
Chorweiler
Blumenberg
Roggendorf/Thenhoven
Worringen
Poll
Ensen
Gremberghoven
Eil
Porz
Urbach
Grengel
Wahnheide
Wahn
Zündorf
Langel
Finkenberg
Humboldt/Gremberg
Kalk
Vingst
Höhenberg
Ostheim
Merheim
Brück
Rath/Heumar
Neubrück
Mülheim
Buchforst
Buchheim
Holweide
Dellbrück
Höhenhaus
Dünnwald
Stammheim
Flittard


In [67]:
# merge dataframes
cologne_venues=cologne_venues.merge(geo, on='neighbourhood')

# drop unnecessary columns to avoid confusion
cologne_venues.drop(['latitude'], 1, inplace = True)
cologne_venues.drop(['longitude'], 1, inplace = True)

#check
cologne_venues

,neighbourhood,neighborhood latitude,neighborhood longitude,venue,venue latitude,venue longitude,venue category,no.,borough,geometry.rings,...,Freie Waldorfschule pp,Förderschule pp,Gesamtschule pp,Grundschule pp,Gymnasium pp,Hauptschule pp,Realschule pp,Schule für Kranke pp,Berufskolleg pp,Weiterbildungskolleg pp
0,Altstadt/Süd,50.930223,6.953587,Motel One Köln-Waidmarkt,50.931557,6.954513,Hotel,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
1,Altstadt/Süd,50.930223,6.953587,Hotel im Wasserturm,50.931745,6.951255,Hotel,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
2,Altstadt/Süd,50.930223,6.953587,Holiday Inn Express,50.930371,6.953661,Hotel,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
3,Altstadt/Süd,50.930223,6.953587,Toscanini,50.925644,6.957099,Italian Restaurant,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
4,Altstadt/Süd,50.930223,6.953587,Metronom,50.930383,6.943511,Pub,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
5,Altstadt/Süd,50.930223,6.953587,Fitness First,50.936639,6.951151,Gym / Fitness Center,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
6,Altstadt/Süd,50.930223,6.953587,Wolkenburg,50.932872,6.944973,Event Space,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
7,Altstadt/Süd,50.930223,6.953587,Apple Schildergasse,50.936552,6.955877,Electronics Store,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
8,Altstadt/Süd,50.930223,6.953587,LEGO Store,50.937042,6.956370,Toy / Game Store,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
9,Altstadt/Süd,50.930223,6.953587,MakiMaki Sushi Green,50.930603,6.947333,Sushi Restaurant,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0


In [68]:
#Check how many venues have been returned

cologne_venues.groupby('neighbourhood').count()

### WARUM MAXIMAL 100 ???

,neighborhood latitude,neighborhood longitude,venue,venue latitude,venue longitude,venue category,no.,borough,geometry.rings,inhabitants/sqm,...,Freie Waldorfschule pp,Förderschule pp,Gesamtschule pp,Grundschule pp,Gymnasium pp,Hauptschule pp,Realschule pp,Schule für Kranke pp,Berufskolleg pp,Weiterbildungskolleg pp
neighbourhood,,,,,,,,,,,,,,,,,,,,,
Altstadt/Nord,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
Altstadt/Süd,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
Bayenthal,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
Bickendorf,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
Bilderstöckchen,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
Blumenberg,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
Bocklemünd/Mengenich,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
Braunsfeld,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
Brück,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17


In [69]:
print('There are {} unique categories.'.format(len(cologne_venues['venue category'].unique())))

There are 261 unique categories.


I define a column that indicates whether a venue is inside the neighbourhood or not.

In [70]:
# create new column for venue inside neighbourhood

cologne_venues.insert(1, 'is_inside', 'null')

def update_inside(row):
    point = Point(row["venue latitude"], row["venue longitude"])
    polygon = Polygon(row["geometry.rings"])
    is_inside = point.within(polygon)
    row['is_inside'] = is_inside
    
    return row

cologne_venues = cologne_venues.apply(update_inside, axis = 1)

cologne_venues # check second column

,neighbourhood,is_inside,neighborhood latitude,neighborhood longitude,venue,venue latitude,venue longitude,venue category,no.,borough,...,Freie Waldorfschule pp,Förderschule pp,Gesamtschule pp,Grundschule pp,Gymnasium pp,Hauptschule pp,Realschule pp,Schule für Kranke pp,Berufskolleg pp,Weiterbildungskolleg pp
0,Altstadt/Süd,True,50.930223,6.953587,Motel One Köln-Waidmarkt,50.931557,6.954513,Hotel,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
1,Altstadt/Süd,True,50.930223,6.953587,Hotel im Wasserturm,50.931745,6.951255,Hotel,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
2,Altstadt/Süd,True,50.930223,6.953587,Holiday Inn Express,50.930371,6.953661,Hotel,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
3,Altstadt/Süd,True,50.930223,6.953587,Toscanini,50.925644,6.957099,Italian Restaurant,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
4,Altstadt/Süd,True,50.930223,6.953587,Metronom,50.930383,6.943511,Pub,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
5,Altstadt/Süd,False,50.930223,6.953587,Fitness First,50.936639,6.951151,Gym / Fitness Center,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
6,Altstadt/Süd,True,50.930223,6.953587,Wolkenburg,50.932872,6.944973,Event Space,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
7,Altstadt/Süd,False,50.930223,6.953587,Apple Schildergasse,50.936552,6.955877,Electronics Store,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
8,Altstadt/Süd,False,50.930223,6.953587,LEGO Store,50.937042,6.956370,Toy / Game Store,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
9,Altstadt/Süd,True,50.930223,6.953587,MakiMaki Sushi Green,50.930603,6.947333,Sushi Restaurant,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0


Now I'll drop every venue that is outside the according neighbourhood to cover all venues within one neighbourhood.

In [71]:
cologne_venues.drop(cologne_venues.loc[cologne_venues['is_inside']== 0].index, inplace=True)
# reset index
cologne_venues=cologne_venues.reset_index(drop = True)

In [72]:
cologne_venues.head()

,neighbourhood,is_inside,neighborhood latitude,neighborhood longitude,venue,venue latitude,venue longitude,venue category,no.,borough,...,Freie Waldorfschule pp,Förderschule pp,Gesamtschule pp,Grundschule pp,Gymnasium pp,Hauptschule pp,Realschule pp,Schule für Kranke pp,Berufskolleg pp,Weiterbildungskolleg pp
0,Altstadt/Süd,True,50.930223,6.953587,Motel One Köln-Waidmarkt,50.931557,6.954513,Hotel,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
1,Altstadt/Süd,True,50.930223,6.953587,Hotel im Wasserturm,50.931745,6.951255,Hotel,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
2,Altstadt/Süd,True,50.930223,6.953587,Holiday Inn Express,50.930371,6.953661,Hotel,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
3,Altstadt/Süd,True,50.930223,6.953587,Toscanini,50.925644,6.957099,Italian Restaurant,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0
4,Altstadt/Süd,True,50.930223,6.953587,Metronom,50.930383,6.943511,Pub,101,Innenstadt,...,0.0,0.000835,0.000418,0.000835,0.001253,0.000418,0.000835,0.0,0.000143,0.0


I check how many venues have been left.

In [73]:
cologne_venues.groupby('neighbourhood').count()

,is_inside,neighborhood latitude,neighborhood longitude,venue,venue latitude,venue longitude,venue category,no.,borough,geometry.rings,...,Freie Waldorfschule pp,Förderschule pp,Gesamtschule pp,Grundschule pp,Gymnasium pp,Hauptschule pp,Realschule pp,Schule für Kranke pp,Berufskolleg pp,Weiterbildungskolleg pp
neighbourhood,,,,,,,,,,,,,,,,,,,,,
Altstadt/Nord,82,82,82,82,82,82,82,82,82,82,...,82,82,82,82,82,82,82,82,82,82
Altstadt/Süd,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
Bayenthal,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
Bickendorf,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
Bilderstöckchen,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
Blumenberg,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Bocklemünd/Mengenich,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
Braunsfeld,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
Brück,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11


In [74]:
print('There are {} uniques categories.'.format(len(cologne_venues['venue category'].unique())))

There are 229 uniques categories.


Now I illustrate that the venues are inside the neighbourhoods on two examples:

Altstadt-Süd und Deutz

In [75]:
map_cologne = folium.Map(location=[latitude_cologne, longitude_cologne], zoom_start=13)

altstadtsüd_poly = folium.PolyLine(geo.loc[0, 'geometry.rings'], color="red", weight=2.5, opacity=1)
altstadtsüd_poly.add_to(map_cologne)   

altstadtsüd_poly = folium.PolyLine(geo.loc[4, 'geometry.rings'], color="blue", weight=2.5, opacity=1)
altstadtsüd_poly.add_to(map_cologne) 


In [76]:
altstadtsüd_data = cologne_venues[['neighbourhood', 'venue', 'venue latitude', 'venue longitude']].copy()
altstadtsüd_data.drop(altstadtsüd_data.loc[altstadtsüd_data['neighbourhood'] != 'Altstadt/Süd'].index, inplace=True)

deutz_data = cologne_venues[['neighbourhood', 'venue', 'venue latitude', 'venue longitude']].copy()
deutz_data.drop(deutz_data.loc[deutz_data['neighbourhood'] != 'Deutz'].index, inplace=True) 

In [77]:
for lat, lng, name in zip(altstadtsüd_data['venue latitude'], altstadtsüd_data['venue longitude'], altstadtsüd_data['venue']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_cologne)
    
for lat, lng, name in zip(deutz_data['venue latitude'], deutz_data['venue longitude'], deutz_data['venue']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_cologne)

map_cologne

To be able to use the data for the clustering of the neighbourhoods later, I'll transform the venue categories into dummies for each neighbourhood.

In [78]:
# one hot encoding
cologne_onehot = pd.get_dummies(cologne_venues[['venue category']], prefix = "", prefix_sep= "")

# add neighborhood column back to dataframe
cologne_onehot['neighbourhood'] = cologne_venues['neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [cologne_onehot.columns[-1]] + list(cologne_onehot.columns[:-1])
cologne_onehot = cologne_onehot[fixed_columns]

cologne_onehot.head()

,neighbourhood,ATM,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Argentinian Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Wine Bar,Zoo,Zoo Exhibit
0,Altstadt/Süd,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Altstadt/Süd,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Altstadt/Süd,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Altstadt/Süd,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Altstadt/Süd,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
cologne_onehot.shape

(1172, 230)

Now I'll group the rows by neighbourhood and by taking the mean of the frequency of occurrence of each category.

In [80]:
cologne_grouped = cologne_onehot.groupby('neighbourhood').mean().reset_index()
cologne_grouped.head()

,neighbourhood,ATM,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Argentinian Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Wine Bar,Zoo,Zoo Exhibit
0,Altstadt/Nord,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012195,0.0,0.0,0.012195,0.0,0.0,0.000000,0.012195,0.0,0.0
1,Altstadt/Süd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.040000,0.0,0.0,0.000000,0.000000,0.0,0.0
2,Bayenthal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.066667,0.000000,0.0,0.0
3,Bickendorf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
4,Bilderstöckchen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [81]:
cologne_grouped.shape

(77, 230)

For a first look, I'll print each neighbourhood along with the top 5 most common venues:

In [82]:
num_top_venues = 5

for hood in cologne_grouped['neighbourhood']:
    print("----"+hood+"----")
    temp = cologne_grouped[cologne_grouped['neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Altstadt/Nord----
               venue  freq
0              Hotel  0.06
1            Brewery  0.05
2              Plaza  0.05
3  German Restaurant  0.04
4               Café  0.04


----Altstadt/Süd----
                venue  freq
0               Hotel  0.16
1   German Restaurant  0.08
2            Creperie  0.04
3        Cocktail Bar  0.04
4  Mexican Restaurant  0.04


----Bayenthal----
                  venue  freq
0             Drugstore  0.13
1           Supermarket  0.13
2        Ice Cream Shop  0.07
3  Gym / Fitness Center  0.07
4       Organic Grocery  0.07


----Bickendorf----
             venue  freq
0      Supermarket  0.27
1  Automotive Shop  0.09
2        Drugstore  0.09
3              Pub  0.09
4        Nightclub  0.09


----Bilderstöckchen----
                venue  freq
0         Supermarket   0.2
1       Auto Workshop   0.1
2  Chinese Restaurant   0.1
3            Bus Stop   0.1
4       Grocery Store   0.1


----Blumenberg----
                venue  freq
0  Light Ra

                        venue  freq
0                 Supermarket  0.17
1                Tram Station  0.17
2                        Park  0.08
3  Financial or Legal Service  0.08
4                Soccer Field  0.08


----Nippes----
                venue  freq
0  Italian Restaurant  0.10
1                 Bar  0.10
2   German Restaurant  0.06
3                Café  0.06
4    Greek Restaurant  0.06


----Ossendorf----
           venue  freq
0    Supermarket  0.21
1   Tram Station  0.11
2       Bus Stop  0.05
3  Hot Dog Joint  0.05
4         Bakery  0.05


----Ostheim----
         venue  freq
0  Supermarket   0.4
1         Lake   0.2
2        Hotel   0.2
3  Pizza Place   0.2
4          ATM   0.0


----Pesch----
                venue  freq
0      Hardware Store  0.17
1          Steakhouse  0.17
2  Italian Restaurant  0.17
3         Supermarket  0.17
4            Bus Stop  0.17


----Poll----
                    venue  freq
0            Tram Station  0.15
1             Beer Garden  0.08
2 

Ultimately, I want to have the 10 most common venues per neighbourhood. Therefore, I'll reuse a function to sort the venues in descending order.

In [83]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now I'll create a new dataframe and display the top 10 venues for each neighbourhood.

In [84]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['neighbourhood'] = cologne_grouped['neighbourhood']

for ind in np.arange(cologne_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cologne_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Altstadt/Nord,Hotel,Brewery,Plaza,Café,German Restaurant,Restaurant,Italian Restaurant,Art Museum,Bakery,Pedestrian Plaza
1,Altstadt/Süd,Hotel,German Restaurant,Pizza Place,Gastropub,Mexican Restaurant,Smoke Shop,Bookstore,Cocktail Bar,Modern European Restaurant,Spanish Restaurant
2,Bayenthal,Drugstore,Supermarket,Burger Joint,Thai Restaurant,German Restaurant,Beer Garden,Bakery,Organic Grocery,Italian Restaurant,Ice Cream Shop
3,Bickendorf,Supermarket,Pub,Burger Joint,Athletics & Sports,Liquor Store,Automotive Shop,Drugstore,Metro Station,Nightclub,Farmers Market
4,Bilderstöckchen,Supermarket,Bus Stop,Chinese Restaurant,Drugstore,Auto Workshop,Grocery Store,Rental Car Location,Beer Garden,Park,Dance Studio


**To get the final data set for the analysis, I'll merge this dataframe to the "main" dataframe of my Cologne data: geo:**

In [85]:
# merge dataframes

geo_analysis = geo.merge(neighbourhoods_venues_sorted, on='neighbourhood')

In [86]:
geo_analysis.head()

,neighbourhood,no.,borough,geometry.rings,inhabitants/sqm,day nurseries pp,latitude,longitude,mv rate,car rate,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Altstadt/Süd,101,Innenstadt,"[[50.936798424501795, 6.940278401301003], [50....",84.698246,0.000430,50.930223,6.953587,46.191579,40.442648,...,Hotel,German Restaurant,Pizza Place,Gastropub,Mexican Restaurant,Smoke Shop,Bookstore,Cocktail Bar,Modern European Restaurant,Spanish Restaurant
1,Neustadt/Süd,102,Innenstadt,"[[50.93748386396194, 6.9264717966942015], [50....",74.182583,0.000725,50.926103,6.946810,41.481053,35.666763,...,Italian Restaurant,Café,Bar,Burger Joint,Cocktail Bar,Vietnamese Restaurant,Middle Eastern Restaurant,Sushi Restaurant,Japanese Restaurant,French Restaurant
2,Altstadt/Nord,103,Innenstadt,"[[50.95008808829535, 6.9589532284153925], [50....",134.388232,0.000493,50.942170,6.954201,68.354804,59.608793,...,Hotel,Brewery,Plaza,Café,German Restaurant,Restaurant,Italian Restaurant,Art Museum,Bakery,Pedestrian Plaza
3,Neustadt/Nord,104,Innenstadt,"[[50.95928768013307, 6.9636858833452315], [50....",121.969731,0.000804,50.948038,6.948013,59.043342,50.140570,...,Italian Restaurant,Café,Bar,Cocktail Bar,Coffee Shop,Park,Movie Theater,Restaurant,Spanish Restaurant,Noodle House
4,Deutz,105,Innenstadt,"[[50.94948568775351, 6.981916500484012], [50.9...",332.813834,0.000826,50.938282,6.980676,69.130062,58.119903,...,Hotel,Park,Bakery,Scenic Lookout,Café,Spa,Supermarket,Museum,Restaurant,Spanish Restaurant


In [87]:
geo_analysis.shape

(77, 109)

In [88]:
geo_analysis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 76
Columns: 109 entries, neighbourhood to 10th Most Common Venue
dtypes: float64(95), int64(1), object(13)
memory usage: 66.2+ KB
